In [1]:
# general use
import os
import scipy as sp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing

sns.set_theme()
from sklearn.model_selection import train_test_split

# for evaluation
from statistics import mean
from sklearn.metrics import f1_score ,accuracy_score, classification_report, confusion_matrix, precision_score, roc_auc_score
from sklearn import metrics


# for current method
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KDTree
from sklearn.model_selection import cross_val_score


In [2]:
data_dirpath = 'dataset'
train_name = 'train.csv'
test_name = 'test.csv'

train_path = os.path.join(data_dirpath, train_name)
test_path = os.path.join(data_dirpath, test_name)
X_train = pd.read_csv(train_path, header=[0])
test_df = pd.read_csv(test_path, header=[0])

print(f'[Default] Number of train data: {X_train.shape[0]}, Number of test data: {test_df.shape[0]}')


lead_map = {'Female': 0 , 'Male': 1}
X_train['Lead'] = X_train['Lead'].map(lead_map).astype(int)

x_data=X_train.loc[:, X_train.columns != 'Lead']
y_data=X_train['Lead']

feature_names = x_data.columns.tolist()


[Default] Number of train data: 1039, Number of test data: 387


In [3]:

X_train = x_data
y_train = y_data





In [4]:
# # adding features: 
# # ratio of dialog length
#------------------------------------------------------------------------------------------
# f2m_ratio = (X_train['Number words male']+1) /( X_train['Number words female']+1 )  

# X_train['female to male ratio'] = f2m_ratio
# print(X_train)

# age_diff = f2m_ratio = X_train['Age Lead'] - X_train['Age Co-Lead'] 
# X_train['Age difference'] = age_diff
#-------------------------------------------------------------------------

#X_train = X_train.drop(columns=['Gross'])#, 'Number words male'])
print(X_train.columns)
#'Number words female' 'Number words male' 'Year'
#['Number words female', 'Total words', 'Number of words lead', 'Difference in words lead and co-lead', 'Number of male actors', 'Year', 'Number of female actors', 'Number words male', 'Gross','Mean Age Male', 'Mean Age Female', 'Age Lead', 'Age Co-Lead']


f2m_ratio = (X_train['Number words male']+1) /( X_train['Number words female']+1 )  

X_train['female to male ratio'] = f2m_ratio
print(X_train.columns)

age_diff = f2m_ratio = X_train['Age Lead'] - X_train['Age Co-Lead'] 
X_train['Age difference'] = age_diff

X_train["YearXGross"] = (np.max(X_train["Year"].values))- X_train["Year"] * X_train["Gross"]
#X_train.drop(['Year', 'Gross'], axis=1, inplace=True)

X_train["words/actors female"] = X_train["Number words female"] / X_train["Number of female actors"]
X_train["words/actors male"] = X_train["Number words male"] / X_train["Number of male actors"]
#X_train.drop(['Number words female', 'Number words male'], axis=1, inplace=True)

X_train["Male/Female Actors Ratio"] = (X_train['Number of male actors']+1) /(X_train['Number of female actors']+1)
X_train["Log Male/Female Actors Ratio"] = np.log(X_train["Male/Female Actors Ratio"])
X_train["Total Actors"] = X_train['Number of male actors']+X_train['Number of female actors']
X_train["LogGross"] = np.log(X_train["Gross"]+1)

 
#X_train.loc[X_train['Lead'] == 1, 'Total Male Words'] = X_train['Number words male'] + X_train['Number of words lead']
#X_train.loc[X_train['Lead'] != 1, 'Total Male Words'] = X_train['Total words'] - (X_train['Number words female'] + X_train['Number of words lead'])

#X_train.loc[X_train['Lead'] == 'Female', 'Total Female Words'] = X_train['Number words female'] + X_train['Number of words lead']
#X_train.loc[X_train['Lead'] != 'Female', 'Total Female Words'] = X_train['Total words'] - (X_train['Number words male'] + X_train['Number of words lead'])
#X_train["Total Male/Female Words Ratio"] = X_train["Total Male Words"] / X_train["Total Female Words"]

#X_train['Words per Male Actor'] = X_train['Total Male Words'] / X_train['Number of male actors']
#X_train['Words per Female Actor'] = X_train['Total Female Words'] / X_train['Number of female actors']
#X_train['Log Words per Male Actor'] = np.log(X_train['Words per Male Actor'])
#X_train['Log Words per Female Actor'] = np.log(X_train['Words per Female Actor'])

#X_train['Log Total Male Words'] = np.log(X_train['Total Male Words'])
#X_train['Log Total Female Words'] = np.log(X_train['Total Female Words'])
#X_train["Log Total Male/Female Words Ratio"] = np.log(X_train["Total Male/Female Words Ratio"])

X_train['Lead Word Dominance'] = X_train['Number of words lead'] / X_train['Total words']

#X_train["Total Male Words Percentage"] = X_train["Total Male Words"]/X_train["Total words"]
#X_train["Total Female Words Percentage"] = X_train["Total Female Words"]/X_train["Total words"]
# X_train["Total Female Words Percentage"] = 1 - X_train["Total Male Words Percentage"]
X_train.head()







Index(['Number words female', 'Total words', 'Number of words lead',
       'Difference in words lead and co-lead', 'Number of male actors', 'Year',
       'Number of female actors', 'Number words male', 'Gross',
       'Mean Age Male', 'Mean Age Female', 'Age Lead', 'Age Co-Lead'],
      dtype='object')
Index(['Number words female', 'Total words', 'Number of words lead',
       'Difference in words lead and co-lead', 'Number of male actors', 'Year',
       'Number of female actors', 'Number words male', 'Gross',
       'Mean Age Male', 'Mean Age Female', 'Age Lead', 'Age Co-Lead',
       'female to male ratio'],
      dtype='object')


,Number words female,Total words,Number of words lead,Difference in words lead and co-lead,Number of male actors,Year,Number of female actors,Number words male,Gross,Mean Age Male,...,female to male ratio,Age difference,YearXGross,words/actors female,words/actors male,Male/Female Actors Ratio,Log Male/Female Actors Ratio,Total Actors,LogGross,Lead Word Dominance
0,1512,6394,2251.0,343,2,1995,5,2631,142.0,51.500000,...,1.739590,-19.0,-281275.0,302.40,1315.500000,0.500000,-0.693147,7,4.962845,0.352049
1,1524,8780,2020.0,1219,9,2001,4,5236,37.0,39.125000,...,3.434098,24.0,-72022.0,381.00,581.777778,2.000000,0.693147,13,3.637586,0.230068
2,155,4176,942.0,787,7,1968,1,3079,376.0,42.500000,...,19.743590,9.0,-737953.0,155.00,439.857143,4.000000,1.386294,8,5.932245,0.225575
3,1073,9855,3440.0,2623,12,2002,2,5342,19.0,35.222222,...,4.974860,10.0,-36023.0,536.50,445.166667,4.333333,1.466337,14,2.995732,0.349061
4,1317,7688,3835.0,3149,8,1988,4,2536,40.0,45.250000,...,1.924886,-3.0,-77505.0,329.25,317.000000,1.800000,0.587787,12,3.713572,0.498829


In [5]:
## normalize all Data: some Data should be normalized together
combined_normalization = [
       ['Number words female','Number words male','Total words', 'Number of words lead','Difference in words lead and co-lead','words/actors female', 'words/actors male'],
       ['Number of male actors','Number of female actors', 'Total Actors'],
       ['Year'],
       ['Gross'],
       ['Mean Age Male', 'Mean Age Female', 'Age Lead', 'Age Co-Lead','Age difference'],
       ['female to male ratio'], 
       ['YearXGross'], 
       ['Male/Female Actors Ratio'],
       ['Log Male/Female Actors Ratio'],
       [ 'LogGross'],
       ['Lead Word Dominance']
 ]
#combined_normalization = [ [col] for col in X_train.columns.tolist() ]

def find_common_mean_norm(column_names, train_set):
    collumn_values = []
    for name in column_names:
        collumn_values.append(train_set[name].values)
    common_array = np.concatenate(collumn_values)
    mean = np.mean(common_array)
    norm = np.linalg.norm(common_array)
    for name in column_names:
        train_set[name] = (train_set[name]-np.min(collumn_values) / (np.max(collumn_values)-np.min(collumn_values))
                           
    return train_set
        
    
for combined in combined_normalization:
     X_train = find_common_mean_norm(combined, X_train)
    
X_train.head()
    
    




SyntaxError: invalid syntax (<ipython-input-5-4c0bea331b0c>, line 27)

In [ ]:
#print(type(X_train1))
columns = X_train.columns
for x in columns:
    print(x)
print(columns.tolist())

In [ ]:
 

# looking at the available metrics
metrics = KDTree.valid_metrics
print(metrics)
# ['euclidean', 'l2', 'minkowski', 'p', 'manhattan', 'cityblock', 'l1', 'chebyshev', 'infinity']


# setting up the classifier KNN
knn = KNeighborsClassifier()
knn.algorithm = 'kd_tree'
knn.metric = metrics[0]
knn.weights ='uniform' #'distance'

#knn.metric = metrics[3]
knn.n_neighbors = 5
knn.n_jobs = multiprocessing.cpu_count()-1
#knn.fit(X_train,y_train)

# #cross_val_score = cross_val_score(knn,X_train,y_train,crossValidationFolds = 5)
# cvs = cross_val_score
# cvs
# cross_val_score.estimator = knn
# cvs.X = X_train
# cross_val_score.y = y_train
# cvs.cv = 5
# cvs.n_jobs = multiprocessing.cpu_count()-1
# cvs.scoring = 'accuracy'
# #cvs.scoring = classification_report
# #cvs.scoring = roc_auc_score
# possible_scores = ['accuracy' , 'f1' , 'roc_auc' , ]

# scores = cross_val_score(estimator = cvs.estimator, X = cvs.X , y = cvs.y , cv = cvs.cv , n_jobs = cvs.n_jobs, scoring = cvs.scoring )


# print(np.mean(scores))

# print(scores)

# cvs.scoring = 'f1'
# scores = cross_val_score(estimator = cvs.estimator, X = cvs.X , y = cvs.y , cv = cvs.cv , n_jobs = cvs.n_jobs, scoring = cvs.scoring )
# print(scores)

##Dropping Features 


all_features = X_train.columns.tolist()




def evaluate_feature(feature, training_set):
    from sklearn.model_selection import StratifiedKFold
    # setting up the data for crossvalidation
    X_train_restricted = training_set.drop(columns=feature) # X_train.drop(columns=feature)
    crossValSegs = StratifiedKFold(n_splits=5,random_state = 42,shuffle = True).split(X_train_restricted,y_train)
    crossValSegs = list(crossValSegs)
    max_accuracy = 0
    max_string = ''
    max_roc_auc = 0
    max_roc_string = ''
    for metric in ['euclidean',  'chebyshev', 'infinity']: #[KDTree.valid_metrics]:
       # print('\n ------------------------------------------------------------ \n')
        knn.metric = metric
        for nneighbours in   [3,4,5,9,10,11]:  # [1,2,3,4,5,7,8,9,11]:
            accuracies = []
            roc_aucs =  []
            knn.n_neighbors = nneighbours
            for train, test in crossValSegs:
                #print(X_train_restricted.values[test],y_train.values[test])
                knn.fit(X_train_restricted.values[train],y_train.values[train])
                probabilities = knn.predict_proba(X_train_restricted.values[test] )
                y_pred = knn.predict(X_train_restricted.values[test])
               # y_pred = np.ones(len(X_train_restricted.values[test]))
                 # evaluate
                accuracy = accuracy_score(y_train.values[test], y_pred)
                accuracy = f1_score(y_train.values[test], y_pred)
                #estimator.predict_proba(X, y)[:, 1]
                roc_auc = roc_auc_score(y_train.values[test], y_pred) #probabilities[:, 1])
                accuracies.append(accuracy)
                roc_aucs.append(roc_auc)
            mean_accuracies = np.mean(accuracies)
            mean_roc_auc = np.mean(roc_aucs)
            current =  ' n =   {} Mean Accuracy  {} Mean ROC Auc Score   {} metric = {} Feature = {}'.format( knn.n_neighbors,mean_accuracies,  mean_roc_auc ,knn.metric, feature)
            if mean_accuracies > max_accuracy: 
                max_accuracy = mean_accuracies
                max_string = current
            #print(current)
            if mean_roc_auc > max_roc_auc: 
                max_roc_auc = mean_roc_auc
                max_roc_string = current
           #print(current)

    print(f'\n\nThe Best Accuracy  Is {max_string}')
    #print(f'\n\nThe Best roc_auc Is {max_roc_string}')
    print(f'maximum accuracy {max_accuracy}')
    return max_accuracy, max_string



    





In [ ]:
# all_features =  x_data.columns.tolist()
feature_acc = []
feature_strings = []
for feature in all_features:
    print(f'current Feature = {feature}')
    max_accuracy, max_string = evaluate_feature([feature], X_train)
    feature_acc.append(max_accuracy)
    feature_strings.append(max_string)
    
normal_accuracy, _ = evaluate_feature([],X_train)    


In [ ]:
fig = plt.figure()

all_features =  X_train.columns.tolist()
feature_acc 
feature_strings 
print(feature_acc, normal_accuracy)

plt.barh(all_features, ((np.array(feature_acc)-normal_accuracy)*100)) #0,len(all_features))
plt.show()


feature_acc = np.array(feature_acc)

# remove all features that decrease the accuracy if they are included
feature_usefull = (feature_acc-normal_accuracy) > 0 

print(feature_acc, feature_usefull)



feature =  [feat for usefull,feat in zip(feature_usefull,all_features) if   usefull]
print(feature)
max_accuracy, max_string = evaluate_feature(feature,X_train)
print('\n\n',max_accuracy, max_string)



        

# Dropping features iteratively:
reducing features one by one 
it has been noticed, that simply dropping all features that impact the accuracy negatively does not yield a favorable result. It drops the accuracy significantly. Only keeping features that impact the accuracy negatively individually however increases the accuracy which is a bit perplexing. Now the strategy is to drop a single feature that impacts the performance the most in a negative direction and then see for the remaining features if there is one that can be removed. I don't expect this to result in a optimal feature combination but it can only improve and not worsen the resulting performance in terms of accuracy on the crossvalidation.

In [ ]:
some_features = all_features =  X_train.columns.tolist()
old_accuracy = normal_accuracy
new_accuracy = normal_accuracy
new_model = ''
old_model = ''
last_feature_removed = ''

done = False

while(not done ):
    old_accuracy = new_accuracy
    old_model = new_model
    feature_acc = []
    feature_strings = []
    for feature in some_features:
        print(f'current Feature eliminated = {feature}')
        max_accuracy, max_string = evaluate_feature([feature],X_train[some_features])
        feature_acc.append(max_accuracy)
        feature_strings.append(max_string)
    ind_min = np.argmin(feature_acc) # find the feature that reduced accuracy the most
    new_accuracy = np.max(feature_acc) # find the new maximum accuracy
    new_model = feature_strings[np.argmax(feature_acc)]
    print(f'new_accuracy = {new_accuracy}, old = {old_accuracy},accuracies = {feature_acc}')
    if(new_accuracy >= old_accuracy): # if the new accuracy is better remove the next feature that negatively influences accuracy the most
        print(f' the last feature removed was {last_feature_removed} , now {some_features[ind_min]} will be removed, accuracy was{old_accuracy} ,\n features = {some_features}')
        last_feature_removed = some_features[ind_min]
        del some_features[ind_min]
        print(f' the last feature')
    else:
        done = True
        some_features.append(last_feature_removed)
    

print(f' The left over Features are: {some_features} , best accuracy: {old_accuracy}, for model: ')
    


In [ ]:
print( set(x_data.columns.tolist()).difference(set(some_features)  ))

print(len(x_data.columns.tolist()))

print(len(some_features))

In [ ]:
columns = X_train.columns.tolist
print(columns)
print(f'current Feature eliminated = {feature}')
max_accuracy, max_string = evaluate_feature(['Number of female actors'],X_train)
print(max_accuracy)
max_accuracy, max_string = evaluate_feature([],X_train)
print(max_accuracy)




In [ ]:
print(X_train[['Difference in words lead and co-lead', 'Number of male actors']])

In [ ]:
print(len(y_data.values), np.sum(y_data.values))